This is a small exercise to efficiently calculate the probability of winning at table tennis.

### Rules:

2 players play against each other, where either player scores a point with some probability at each time step. The first player to reach 21 wins. If there's a difference of less than 2 points, the players keep playing until a player wins by getting a lead of 2 points.

### Analysis

For a concrete example, I'll do the below analysis for the case where one player has a 60% chance of winning a point.

If we consider playing 40 points, winning exactly 20 points will result in a tie and winning 21 or more points would result in a win. Of course, we stop playing at 21, but continuing playing up to 40 points after winning, or losing, can't change the outcome, since the highest score the other player can reach is 19, so this also accurately calculates the probability of winning had we stopped at 21.

We can use the calculation below to determine the probability of winning from 20-20.

```
p(win from equal) = p(2W) + p(WL or LW) * p(win from equal)             Since WL and LW both lead back to an equal score, and 2L is a loss
p(win from equal) - p(WL or LW) * p(win from equal) = p(2W)
p(win from equal) * (1 - p(WL or LW)) = p(2W)
p(win from equal) = p(2W) / (1 - p(WL or LW))
p(win from equal) = p(2W) / (p(2W) + p(2L))                             WL, LW, WW and LL are the only possibilities for 2 games, and they're disjoint,
                                                                        thus p(2W) + p(2L) = 1 - p(WL or LW)
p(win from equal) = 0.6 * 0.6 / (0.6 * 0.6 + 0.4 * 0.4)                 Since p(2W) = p(w)^2 = 0.6 * 0.6, and the same for p(2L)
p(win from equal) = 0.6923...
```

So we have `p(win) = p(X >= 21) + p(X = 20) * p(win from equal) = 0.8702 + 0.0554 * 0.6923 = 90.9%`.

For validating these results, I also wrote a more brute-force method, which exhaustively calculates the probabilities of reaching every pair of scores. Starting with 0-0 with a probability of 1, we use that to determine the probabilities of 1-0 => `1*0.6` and 0-1 => `1*0.4`, from 1-0 we get 2-0 => `1*0.6*0.6` and 1-1 => `1*0.6*0.4`, and so on, summing duplicated probabilities. We repeat this and we add any winning state (won at 21) to the probability of winning and just ignore losing states (lost at 21).

This leaves us with 20-20, which we calculate separately, as above.

As expected, this also gives a probability of about `90.9%` of winning the game.

In [1]:
from random import *
from collections import Counter
from decimal import *
from scipy.stats import binom

def will_win(p_win: float, win_score: float) -> bool:
    p1 = 0
    p2 = 0
    while True:
        if random() < p_win:
            p1 += 1
        else:
            p2 += 1
        if p1 >= win_score and p2 + 2 <= p1:
            return True
        if p2 >= win_score and p1 + 2 <= p2:
            return False


def win_prob_brute_force(p_win: float, win_score: float, trials: int) -> float:
    wins = 0
    for _ in range(trials):
        if will_win(p_win, win_score):
            wins += 1
    return wins / trials


def win_probability_manual(p_win: float, win_score: float) -> float:
    """ Manually calculate binomial distribution """
    probabilities = Counter({(0,0): 1})
    p_loss = 1 - p_win
    p_win_overall = 0

    while probabilities:
        probs_temp = Counter()
        for key,value in probabilities.items():
            if key[0] == win_score and key[1] + 2 <= key[0]:
                p_win_overall += value
            elif key[1] == win_score and key[0] + 2 <= key[1]:
                pass # we don't care about losses
            elif key[0] == key[1] == win_score - 1:
                p_tie = value
            else:
                probs_temp[(key[0] + 1, key[1])] += value * p_win
                probs_temp[(key[0], key[1] + 1)] += value * p_loss
        probabilities = probs_temp

    p_win_from_equal = p_win * p_win / (p_win * p_win + p_loss * p_loss)

    print("Interim probability - initial win:", p_win_overall)
    print("Interim probability - initial tie:", p_tie)
    print("Interim probability - win from equal:", p_win_from_equal)

    p_win_overall += p_tie * p_win_from_equal
    return p_win_overall


def win_probability(p_win: float, win_score: float) -> float:
    p_loss = 1 - p_win
    tie_score = win_score - 1
    p_win_overall = 1 - binom.cdf(tie_score, 2 * tie_score, p_win)
    p_tie = binom.pmf(tie_score, 2 * tie_score, p_win)
    p_win_from_equal = p_win * p_win / (p_win * p_win + p_loss * p_loss)
    p_win_overall += p_tie * p_win_from_equal
    return p_win_overall


print("Probability (manual):", win_probability_manual(0.6, 21))
print("Probability:", win_probability(0.6, 21))
print("Probability (brute force):", win_prob_brute_force(0.6, 21, 1000000))

Interim probability - initial win: 0.8702342941780972
Interim probability - initial tie: 0.05541414906498898
Interim probability - win from equal: 0.6923076923076923
Probability (manual): 0.9085979358384741
Probability: 0.9085979358384739
Probability (brute force): 0.908663
